In [ ]:
import numpy as np
import pandas as pd
import random as rd

rd.seed(123)

In [ ]:
# import all the files

donationsDF = pd.read_csv('Donations.csv')
donorsDF = pd.read_csv('Donors.csv')
projectsDF = pd.read_csv('Projects.csv')
schoolsDF = pd.read_csv('Schools.csv')
resourcesDF = pd.read_csv('Resources.csv')

In [3]:
donationsDF.head(2)

,Project ID,Donation ID,Donor ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date
0,000009891526c0ade7180f8423792063,688729120858666221208529ee3fc18e,1f4b5b6e68445c6c4a0509b3aca93f38,No,178.37,11,2016-08-23 13:15:57
1,000009891526c0ade7180f8423792063,dcf1071da3aa3561f91ac689d1f73dee,4aaab6d244bf3599682239ed5591af8a,Yes,25.00,2,2016-06-06 20:05:23


In [4]:
donorsDF.head(2)

,Donor ID,Donor City,Donor State,Donor Is Teacher,Donor Zip
0,00000ce845c00cbf0686c992fc369df4,Evanston,Illinois,No,602
1,00002783bc5d108510f3f9666c8b1edd,Appomattox,other,No,245


In [5]:
projectsDF.head(2)

,Project ID,School ID,Teacher ID,Teacher Project Posted Sequence,Project Type,Project Title,Project Essay,Project Short Description,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
0,7685f0265a19d7b52a470ee4bac883ba,e180c7424cb9c68cb49f141b092a988f,4ee5200e89d9e2998ec8baad8a3c5968,25,Teacher-Led,Stand Up to Bullying: Together We Can!,Did you know that 1-7 students in grades K-12 ...,Did you know that 1-7 students in grades K-12 ...,"My students need 25 copies of ""Bullying in Sch...",Applied Learning,"Character Education, Early Development",Grades PreK-2,Technology,361.80,2013-01-01,2013-05-30,Fully Funded,2013-01-11
1,f9f4af7099061fb4bf44642a03e5c331,08b20f1e2125103ed7aa17e8d76c71d4,cca2d1d277fb4adb50147b49cdc3b156,3,Teacher-Led,Learning in Color!,"Help us have a fun, interactive listening cent...","Help us have a fun, interactive listening cent...","My students need a listening center, read alon...","Applied Learning, Literacy & Language","Early Development, Literacy",Grades PreK-2,Technology,512.85,2013-01-01,2013-05-31,Expired,NaN


In [6]:
donorsDF_merged = donorsDF.merge(donationsDF, on = 'Donor ID', how = 'left')

In [7]:
donorsDF_merged.head(1)

,Donor ID,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date
0,00000ce845c00cbf0686c992fc369df4,Evanston,Illinois,No,602,5bab6101eed588c396a59f6bd64274b6,598691d82438952e2e4f3ed50531fd2a,Yes,50.0,1.0,2013-12-17 21:47:14


In [23]:
# Get the count of donors based on number of donations made
df_temp1 = donorsDF_merged.groupby( ['Donor ID']).size().reset_index(name='Num Donations')

df_temp1.head(2)

,Donor ID,Num Donations
0,00000ce845c00cbf0686c992fc369df4,1
1,00002783bc5d108510f3f9666c8b1edd,1


In [17]:
#df_temp1[df_temp1['count']>1]

df_temp2 = df_temp1[df_temp1['Num Donations'] > 1] # removing all the single-donors

print("We got rid of %d single-donors." % (df_temp1.shape[0] - df_temp2.shape[0]))

We got rid of 1569968 single-donors


In [24]:
df_temp2.head()

print("Remaining dataframe shape: ", df_temp2.shape)

Remaining dataframe shape:  (552672, 2)


In [33]:
donorsDF_merged.shape

(4785578, 11)

In [78]:
uniqDonorIDList = list(df_temp2['Donor ID'])

In [79]:
df_final = donorsDF_merged[donorsDF_merged['Donor ID'].isin(uniqDonorIDList)]

# filtered_df = df[df['Country Name'].isin(country_list)] 


# df_final = df_temp2.merge(donorsDF_merged, on = 'Donor ID', how = 'left')

df_final.shape

(3215610, 11)

In [80]:
df_final = df_final.drop_duplicates(subset = ['Donation ID']) # removes duplicates

df_final.shape

(3215570, 11)

In [84]:
df_final.head()

,Donor ID,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date
2,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953,2f53e5f31890e647048ac217cda3b83f,7831f66a25935db8b7424170355edabf,No,26.00,10.0,2017-05-02 12:34:09
3,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953,2f53e5f31890e647048ac217cda3b83f,ee15bb7f179142fdff3f24cff8e88483,No,100.00,9.0,2017-04-01 01:08:50
4,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953,2f7996f08052785e9b146f72c0c4990d,c73207489356a7bcf3ef0b8660c61b04,Yes,13.75,5.0,2017-01-16 15:46:57
5,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953,64f54f1efcbeb986114a7a13e6b27257,e3162fdec64ed91de1aa096cf75ba032,Yes,100.00,3.0,2017-01-16 14:20:10
6,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953,75131d2e94930082aa8ed1e4cd4d21da,234b64bba2dbaf1419492befcf767425,Yes,100.00,8.0,2017-03-13 18:37:24


In [82]:
# df_final[df_final['Donor ID'] == '00005f52c98eeaf92b2414a352b023a4']

In [83]:
projectsDF.head(2)

,Project ID,School ID,Teacher ID,Teacher Project Posted Sequence,Project Type,Project Title,Project Essay,Project Short Description,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
0,7685f0265a19d7b52a470ee4bac883ba,e180c7424cb9c68cb49f141b092a988f,4ee5200e89d9e2998ec8baad8a3c5968,25,Teacher-Led,Stand Up to Bullying: Together We Can!,Did you know that 1-7 students in grades K-12 ...,Did you know that 1-7 students in grades K-12 ...,"My students need 25 copies of ""Bullying in Sch...",Applied Learning,"Character Education, Early Development",Grades PreK-2,Technology,361.80,2013-01-01,2013-05-30,Fully Funded,2013-01-11
1,f9f4af7099061fb4bf44642a03e5c331,08b20f1e2125103ed7aa17e8d76c71d4,cca2d1d277fb4adb50147b49cdc3b156,3,Teacher-Led,Learning in Color!,"Help us have a fun, interactive listening cent...","Help us have a fun, interactive listening cent...","My students need a listening center, read alon...","Applied Learning, Literacy & Language","Early Development, Literacy",Grades PreK-2,Technology,512.85,2013-01-01,2013-05-31,Expired,NaN


In [85]:
df_final = df_final.merge(projectsDF, on = "Project ID", how = "left")

df_final.head(2)

,Donor ID,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,...,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
0,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953,2f53e5f31890e647048ac217cda3b83f,7831f66a25935db8b7424170355edabf,No,26.0,10.0,...,My students need books to inspire them to be a...,"History & Civics, Literacy & Language","Civics & Government, Literacy",Grades 6-8,Books,373.53,2017-04-01,2017-08-01,Fully Funded,2017-05-12
1,00002d44003ed46b066607c5455a999a,Winton,California,Yes,953,2f53e5f31890e647048ac217cda3b83f,ee15bb7f179142fdff3f24cff8e88483,No,100.0,9.0,...,My students need books to inspire them to be a...,"History & Civics, Literacy & Language","Civics & Government, Literacy",Grades 6-8,Books,373.53,2017-04-01,2017-08-01,Fully Funded,2017-05-12


In [86]:
# df_final[df_final['Donor ID'] == '00005f52c98eeaf92b2414a352b023a4']

,Donor ID,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,...,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
18,00005f52c98eeaf92b2414a352b023a4,Villanova,Pennsylvania,No,190,1e86803ae2a4fc9831712f6e2a4c0a5f,70ba9ef18a65099d30a4a9d78de4ed48,Yes,10.0,2.0,...,"My students need Read-Along stories, books, an...","Literacy & Language, Special Needs","Literacy, Special Needs",Grades 6-8,Other,390.47,2013-10-06,2014-01-28,Fully Funded,2014-01-23
19,00005f52c98eeaf92b2414a352b023a4,Villanova,Pennsylvania,No,190,75ad8c8c18c370b8e1f829d2887a3b0b,9869d0d720935c1c0ca4454f7c0b2d5f,Yes,10.0,1.0,...,"My students need a reading area or ""literature...","Math & Science, Literacy & Language","Environmental Science, Literacy",Grades 3-5,Books,744.67,2013-08-08,2013-12-03,Fully Funded,2013-09-20


In [92]:
columnsList = ['Donor ID', 'Donation Received Date', 'Donor City', 'Donor State',
       'Donor Is Teacher', 'Donor Zip', 'Project ID', 'Donation ID',
       'Donation Included Optional Donation', 'Donation Amount',
       'Donor Cart Sequence', 'School ID',
       'Teacher ID', 'Teacher Project Posted Sequence', 'Project Type',
       'Project Title', 'Project Essay', 'Project Short Description',
       'Project Need Statement', 'Project Subject Category Tree',
       'Project Subject Subcategory Tree', 'Project Grade Level Category',
       'Project Resource Category', 'Project Cost', 'Project Posted Date',
       'Project Expiration Date', 'Project Current Status',
       'Project Fully Funded Date']

df_final = df_final[columnsList]

df_final = df_final.sort_values(by = ['Donor ID', 'Donation Received Date'])



,Donor ID,Donation Received Date,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,...,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
9,00002d44003ed46b066607c5455a999a,2016-10-25 20:15:11,Winton,California,Yes,953,e2beb818569f66adaa4ced21ca299ac6,08ed72ce14c548e8131a7dd7b8561988,Yes,10.00,...,My students need a classroom library so they c...,"History & Civics, Literacy & Language","History & Geography, Literacy",Grades 6-8,Books,555.28,2016-10-23,2017-02-22,Fully Funded,2016-12-26
10,00002d44003ed46b066607c5455a999a,2017-01-16 01:11:20,Winton,California,Yes,953,eb6d91cbeab5037ca2f45fc3f6a4de8c,158f00637416b7c41fcde429b10c3c41,Yes,15.51,...,"My students need deodorant, soap, toothbrush a...","Warmth, Care & Hunger","Warmth, Care & Hunger",Grades 6-8,Supplies,305.51,2017-01-16,2017-05-12,Fully Funded,2017-01-16


In [ ]:
for index, row in df_final.iterrows(): # creates a dictionary for county-state pairs
    if df_final['']

In [96]:
df_final.tail(10)

,Donor ID,Donation Received Date,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,Donation Amount,...,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
3215561,ffff486ac82bf347e51e39f5a72af66d,2016-08-18 08:50:15,San Francisco,California,Yes,941,368fb9690430abc493df7f9ac8ca9434,4ade3484c21c0faf174a4650870de899,Yes,25.00,...,My students need ten Chromebooks.,"Literacy & Language, Math & Science","Literacy, Mathematics",Grades 3-5,Technology,2495.51,2016-08-16,2016-12-15,Fully Funded,2016-08-23
3215565,ffff4d8dadd5e8848fefe2a6c52daac9,2014-10-27 09:34:21,Jackson,Georgia,No,302,5f8bc73d65a501f6f0559759335822e8,d3cff8fbe23213dfc3aaa8f49c2b9dd3,Yes,25.00,...,"My students need ear guards, singlets, knee pa...",Health & Sports,"Health & Wellness, Team Sports",Grades 9-12,Supplies,980.11,2014-10-14,2015-02-12,Fully Funded,2014-12-06
3215563,ffff4d8dadd5e8848fefe2a6c52daac9,2015-01-26 11:50:38,Jackson,Georgia,No,302,008dbbeb2de7625ad8ecaef850f1b462,0bd3d0ecdfb84752a3ee0f01f5b65906,Yes,6.32,...,My students need a listening center. My studen...,"Applied Learning, Literacy & Language","Character Education, Literacy",Grades 3-5,Other,600.31,2014-10-01,2015-01-30,Fully Funded,2015-01-26
3215564,ffff4d8dadd5e8848fefe2a6c52daac9,2015-02-27 11:14:31,Jackson,Georgia,No,302,0fc976bbbc072203979f59ece0c9f8f6,33b655862e6bba43de52e54e4ffe4552,Yes,25.00,...,My students need training aids such as parachu...,Health & Sports,"Health & Wellness, Team Sports",Grades 9-12,Supplies,412.62,2015-02-24,2015-06-24,Fully Funded,2015-02-27
3215566,ffff4d8dadd5e8848fefe2a6c52daac9,2016-01-15 13:40:51,Jackson,Georgia,No,302,ebcbab529d58db06e4bf3b2bfe6c3468,7e525a00b845e49555b29ee7a93aefa3,Yes,10.00,...,My students need six Gill National Starting Bl...,Health & Sports,Team Sports,Grades 6-8,Other,2809.89,2015-10-16,2016-02-09,Expired,NaN
3215568,ffff8bee61b72c484b10e43aa9e35bc9,2016-09-01 19:03:42,Waltham,Massachusetts,Yes,24,5b11d16bbe347b9d842b02635f02ad86,8f31a87cb7a6ff1ec10b636660c0353f,Yes,75.00,...,My students need supplies to conduct their sci...,Math & Science,Health & Life Science,Grades 9-12,Supplies,500.31,2016-09-01,2016-12-31,Fully Funded,2016-09-09
3215567,ffff8bee61b72c484b10e43aa9e35bc9,2016-09-01 19:08:53,Waltham,Massachusetts,Yes,24,5b11d16bbe347b9d842b02635f02ad86,af301331ed8a3b89ae70330c5e6a9bb3,Yes,99.00,...,My students need supplies to conduct their sci...,Math & Science,Health & Life Science,Grades 9-12,Supplies,500.31,2016-09-01,2016-12-31,Fully Funded,2016-09-09
3215569,ffff8bee61b72c484b10e43aa9e35bc9,2017-11-28 14:33:00,Waltham,Massachusetts,Yes,24,9d1b86c354015c49cbdf8808245aab8c,a30233e31e1c0a72425495e8917600a2,Yes,10.00,...,My students need headphones to listen to onlin...,"Math & Science, Literacy & Language","Applied Sciences, Literature & Writing",Grades 6-8,Classroom Basics,461.81,2017-11-28,2018-03-28,Fully Funded,2018-01-25
3215570,ffffad3f78f2b13ce571c709dd049cd3,2014-12-09 08:33:53,Houston,Mississippi,No,770,3c2544dec2bfd9e5f62f39f183c7ea55,f91671758ed9795e17c9f9cb2b81a96e,Yes,25.00,...,My students need chrome books in our classroom...,"Literacy & Language, Special Needs","Literature & Writing, Special Needs",Grades 3-5,Technology,1949.94,2014-11-19,2015-03-18,Expired,NaN
3215571,ffffad3f78f2b13ce571c709dd049cd3,2015-10-14 08:48:06,Houston,Mississippi,No,770,a56887997b5c9ab80abdc3c7a8210cea,092d3e384a61d157a4c157935219cc30,Yes,100.00,...,My students need an LCD projector with a mount...,"Literacy & Language, Math & Science","Literacy, Mathematics",Grades PreK-2,Technology,944.89,2015-09-20,2016-01-16,Fully Funded,2016-01-07


In [87]:
df = df_final.sample(50000)

In [88]:
df.columns

Index(['Donor ID', 'Donor City', 'Donor State', 'Donor Is Teacher',
       'Donor Zip', 'Project ID', 'Donation ID',
       'Donation Included Optional Donation', 'Donation Amount',
       'Donor Cart Sequence', 'Donation Received Date', 'School ID',
       'Teacher ID', 'Teacher Project Posted Sequence', 'Project Type',
       'Project Title', 'Project Essay', 'Project Short Description',
       'Project Need Statement', 'Project Subject Category Tree',
       'Project Subject Subcategory Tree', 'Project Grade Level Category',
       'Project Resource Category', 'Project Cost', 'Project Posted Date',
       'Project Expiration Date', 'Project Current Status',
       'Project Fully Funded Date'],
      dtype='object')

In [89]:
columnsList = ['Donor ID', 'Num Donations', 'Donation Received Date', 'Donor City', 'Donor State',
       'Donor Is Teacher', 'Donor Zip', 'Project ID', 'Donation ID',
       'Donation Included Optional Donation', 'Donation Amount',
       'Donor Cart Sequence', 'School ID',
       'Teacher ID', 'Teacher Project Posted Sequence', 'Project Type',
       'Project Title', 'Project Essay', 'Project Short Description',
       'Project Need Statement', 'Project Subject Category Tree',
       'Project Subject Subcategory Tree', 'Project Grade Level Category',
       'Project Resource Category', 'Project Cost', 'Project Posted Date',
       'Project Expiration Date', 'Project Current Status',
       'Project Fully Funded Date']

# df = df[cols]

In [69]:
df = df[columnsList]

In [74]:
df = df.sort_values(by = ['Donor ID', 'Donation Received Date'])

df[df['Donor ID'] == '00005f52c98eeaf92b2414a352b023a4']

,Donor ID,Num Donations,Donation Received Date,Donor City,Donor State,Donor Is Teacher,Donor Zip,Project ID,Donation ID,Donation Included Optional Donation,...,Project Need Statement,Project Subject Category Tree,Project Subject Subcategory Tree,Project Grade Level Category,Project Resource Category,Project Cost,Project Posted Date,Project Expiration Date,Project Current Status,Project Fully Funded Date
19,00005f52c98eeaf92b2414a352b023a4,2,2013-09-19 13:58:55,Villanova,Pennsylvania,No,190,75ad8c8c18c370b8e1f829d2887a3b0b,9869d0d720935c1c0ca4454f7c0b2d5f,Yes,...,"My students need a reading area or ""literature...","Math & Science, Literacy & Language","Environmental Science, Literacy",Grades 3-5,Books,744.67,2013-08-08,2013-12-03,Fully Funded,2013-09-20


In [75]:
donationsDF[donationsDF['Donor ID'] == '00005f52c98eeaf92b2414a352b023a4']

,Project ID,Donation ID,Donor ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date
559265,1e86803ae2a4fc9831712f6e2a4c0a5f,70ba9ef18a65099d30a4a9d78de4ed48,00005f52c98eeaf92b2414a352b023a4,Yes,10.0,2,2014-01-21 17:59:43
2151243,75ad8c8c18c370b8e1f829d2887a3b0b,9869d0d720935c1c0ca4454f7c0b2d5f,00005f52c98eeaf92b2414a352b023a4,Yes,10.0,1,2013-09-19 13:58:55
